Case 6
Fabian Brock
i6248959

# LIBOR Market Model

## Discount bond prices

In [136]:
import numpy as np
from scipy.stats import norm
import pandas as pd

In [137]:
annual_interest_rate = 0.03  # 3% interest rate
maturities = range(1, 11)  # Maturities from 1 to 9 years

# using the formula P(0, T) = 1 / (1 + r)^T
bond_prices = {T: 1 / ((1 + annual_interest_rate) ** T) for T in maturities}
bond_prices

{1: 0.970873786407767,
 2: 0.9425959091337544,
 3: 0.9151416593531596,
 4: 0.8884870479156888,
 5: 0.8626087843841639,
 6: 0.8374842566836542,
 7: 0.8130915113433536,
 8: 0.7894092343139355,
 9: 0.7664167323436267,
 10: 0.7440939148967249}

In [138]:
initial_forward_rate = 0.03
sigma = 0.01
num_maturities = 9  # f0 to f9
num_paths = 10000
time_step = 1 # deltaT
total_time = 10  # sim time

# timepoints
num_steps = int(total_time / time_step)
time_points = np.linspace(0, total_time, num_steps + 1)

# init forward rates matrix
forward_rates = np.zeros((num_paths, num_maturities, num_steps + 1))
forward_rates[:, :, 0] = initial_forward_rate

# MC
np.random.seed(0)
for path in range(num_paths):
    for step in range(num_steps):
        dt = time_points[step + 1] - time_points[step]
        dW = np.random.normal(0, np.sqrt(dt), num_maturities)
        forward_rates[path, :, step + 1] = forward_rates[path, :, step] + sigma * dW

# avg forward rates of all paths
average_forward_rates = forward_rates.mean(axis=0)

# avg forward rates at year end
average_forward_rates_at_year_end = average_forward_rates[:, ::int(1/time_step)]
# avg per column
average_forward_rates = average_forward_rates_at_year_end.mean(axis=0)
average_forward_rates

array([0.03      , 0.02997317, 0.02996082, 0.02995905, 0.03002102,
       0.03000746, 0.03009609, 0.03009348, 0.03009618, 0.03012247,
       0.03017021])

In [139]:
# d10 as numeraire
num_maturities = 10  # f0 to f9
num_steps = num_maturities
time_step = 1  # dt = 1
sigma = 0.01

# init forward rates matrix
forward_rates_mc = np.zeros((num_paths, num_maturities, num_steps + 1))
forward_rates_mc[:, :, 0] = initial_forward_rate

# MC q10
np.random.seed(0)
for path in range(num_paths):
    for step in range(num_steps):
        dW = np.random.normal(0, np.sqrt(time_step), num_maturities)
        forward_rates_mc[path, :, step + 1] = forward_rates_mc[path, :, step] + sigma * dW


mc_discount_bond_values = np.zeros((num_maturities, num_steps + 1))

# now value and average over paths
for i in range(num_maturities):
    for path in range(num_paths):
        # "discount factor"
        discount_factor = np.prod(1 / (1 + time_step * forward_rates_mc[path, :i+1, i]))
        mc_discount_bond_values[i] += discount_factor

    # avg
    mc_discount_bond_values[i] /= num_paths

# get at t=0
mc_discount_values_at_t0 = mc_discount_bond_values[:, 0]

In [140]:
# convert bond prices values to numpy array
libor = np.array(list(bond_prices.values()))
both = pd.DataFrame(np.vstack([libor, mc_discount_values_at_t0]).T, columns=["LIBOR", "MC"])
both["Difference"] = both.LIBOR - both.MC
both

,LIBOR,MC,Difference
0,0.970874,0.970874,8.570922e-14
1,0.942596,0.942832,-2.361074e-04
2,0.915142,0.915634,-4.927686e-04
3,0.888487,0.889682,-1.195094e-03
4,0.862609,0.864407,-1.798257e-03
5,0.837484,0.839717,-2.232433e-03
6,0.813092,0.815885,-2.793813e-03
7,0.789409,0.793326,-3.916634e-03
8,0.766417,0.771527,-5.109835e-03
9,0.744094,0.749409,-5.315102e-03


## Gaussian Swaption Formulas

In [141]:
def gauss_payer_swaption(principal, strike, volatility, option_maturity, current_time, discount_factors):

    sum_discount_factors = sum(discount_factors)
    forward_rate = (discount_factors[0] - discount_factors[-1]) / sum_discount_factors
    d = (forward_rate - strike) / (volatility * np.sqrt(option_maturity - current_time))
    phi_d = np.exp(-0.5 * d**2) / np.sqrt(2 * np.pi)

    swaption_price = principal * sum_discount_factors * ((forward_rate - strike) * norm.cdf(d) + volatility * np.sqrt(option_maturity - current_time) * phi_d)
    return swaption_price

In [142]:
# Parameters
principal = 1
volatility = 0.01  # gaussian
option_maturity = 10
current_time = 0
maturities = np.arange(1, 10)  # maturieites from 1 to 9
strikes = np.arange(0.01, 0.06, 0.01)  # strike rates from 1% to 5%
annual_interest_rate = 0.03  # term strucuture 3%
num_paths = 10000

# "discount facotrs"
#discount_factors = [np.exp(-annual_interest_rate * t) for t in range(1, option_maturity + 1)]
discount_factors = [ 1 / ((1 + annual_interest_rate) ** T) for T in maturities]


# swaption prices
swaption_prices = np.array([
    [gauss_payer_swaption(principal, strike, volatility, option_maturity, current_time, discount_factors[:maturity])
     for strike in strikes] for maturity in maturities])


# make dataframe out of swaption prices
swaption_prices_df = pd.DataFrame(swaption_prices, index=maturities, columns=strikes)
swaption_prices_df


,0.01,0.02,0.03,0.04,0.05
1,0.008001,0.004911,0.002818,0.001505,0.000746
2,0.028986,0.019472,0.012320,0.007305,0.004040
3,0.051075,0.035266,0.022997,0.014087,0.008067
4,0.072929,0.051026,0.033763,0.021012,0.012239
5,0.094303,0.066494,0.044377,0.027878,0.016402
6,0.115130,0.081593,0.054764,0.034616,0.020502
7,0.135392,0.096299,0.064893,0.041199,0.024517
8,0.155090,0.110605,0.074756,0.047616,0.028435
9,0.174230,0.124512,0.084350,0.053863,0.032254


In [143]:
gauss_payer_swaption(principal, 0.03, volatility, option_maturity, current_time, discount_factors[:9])

0.08435028650795875

In [144]:
strike = 0.03
# gaussian swaptions
gaussian_swaption_prices = pd.DataFrame(index=maturities)
for maturity in maturities:

        price = gauss_payer_swaption(principal, strike, volatility, maturity, 0, mc_discount_values_at_t0[:maturity])
        gaussian_swaption_prices.at[maturity, strike] = price

# lmm swaption using MC
lmm_swaption_prices = pd.DataFrame(index=maturities)
for maturity in maturities:

        payoffs = []
        for path in range(num_paths):
            forward_rate_at_maturity = forward_rates_mc[path, maturity - 1, maturity]
            payoff = max(forward_rate_at_maturity - strike, 0) * principal
            discounted_payoff = payoff * mc_discount_values_at_t0[maturity - 1]
            payoffs.append(discounted_payoff)
        lmm_price = np.mean(payoffs)
        lmm_swaption_prices.at[maturity, strike] = lmm_price

# merge dfs and rename cols
both = pd.merge(gaussian_swaption_prices, lmm_swaption_prices, left_index=True, right_index=True)
both.columns = ["Gaussian", "LMM"]
both["Difference"] = both.Gaussian - both.LMM
both



,Gaussian,LMM,Difference
1,0.000004,0.003900,-0.003897
2,0.001912,0.005248,-0.003336
3,0.008201,0.006474,0.001727
4,0.016925,0.007024,0.009902
5,0.027230,0.007651,0.019579
6,0.038713,0.008295,0.030418
7,0.051009,0.008689,0.042321
8,0.063758,0.008866,0.054892
9,0.077042,0.009356,0.067687


I have no idea why these are so far apart, I guess the guassian approximation is not very good (or I made a mistake). Maybe its because of the euler discretization, especially only taking 1y timesteps?